import python modules

In [1]:
from scipy import stats
import numpy as np

### Table of Contents
1. [Background](#background)<br><br>
2. [Hypothesis testing with one sample](#onesamp)<br>
    2a. [if population sigma is unknown -> t distribution](#t_1s)<br>
    2b. [if population sigma is known -> normal distribution](#norm_1s)<br>
    2c. [for a poplation proportion -> binomial as normal](#bi_1s)<br><br>
3. [Hypothesis testing with two independent samples](#twosamp_ind)<br>
    3a. [Welch](#welch)<br>
    3b. [Cohen's d](#cohen_d)<br>
    3c. [Two independent proportions](#ind_pro)<br><br>
4. [Hypothesis testing with two dependent samples](#twosamp_dep)<br><br>
5. [Chi squared test](#chi_sq)<br>
    5a. [Goodness-of-fit](#chi_fit)<br>
    5b. [Independence](#chi_ind)<br>
    5c. [Homogeneity](#chi_homo)<br>
    5d. [Single variance](#chi_var)<br>

### Background<a class="anchor" id="background"></a>

A **hypothesis test** is a statistical inference that makes a decision about the validity of an assertion which has been translated into a statistic. It considers a **null hypothesis** $H_0$ against an **alternative hypothesis** $H_1$ or $H_a$. The null hypothesis represents some default position that the results of the test will either rejected or failed to reject in favor of the alternative hypothesis which is the opposite of the null.
> $H_0$ null hypothesis<br>
$H_1$ alternative hypothesis 

Before data collection, a **signifiance level**, $\alpha$, is chosen. It represents the probability that the rest will incorrectly reject the null hypothesis (that is reject it even though it is true). After data collection, the signifiance level is compared against the **p-value** which is the probability of observing a test statistic at least as extreme as the one actually obsevered given that null hypothesis is true. 
>If p-value < $\alpha$ then the null hypothesis is rejected at significance level $\alpha$.

The **signifiance** of a test is the probability of making a **type 1 error** or **false positive**. Also of interest is the probability failing to reject the null hypothesis when it is false $\beta$. That is a **type 2 error** or **false negative**. This is more commonly expression as the probability of *not* making a type 2 error or the **power** of the test $1 - \beta$.<br>
>$\alpha$ = P(type 1)<br>
$\beta$ = P(type 2)<br><br>
signifiance = $\alpha$<br>
power = $1 - \beta$

Tests can be left-tailed, right-tailed or two-tailed depending on the hypothesis:<br>
>Left-tailed: p-value is in the left tail of the distribution
$H_0$: $\mu >= x$ and $H_1$: $\mu < x$<br>
Right-tailed: p-value is in the right tail of the distribution
$H_0$: $\mu <= x$ and $H_1$: $\mu > x$<br>
Two-tailed: p-value split evenly between both tails of the distribution
$H_0$: $\mu = x$ and $H_1$: $\mu <> x$<br> 

### Hypothesis testing with one sample<a class="anchor" id="onesamp"></a>



#### single population mean -  unknown population standard deviation <a class="anchor" id="t_1s"></a> 

When the population standard deviation is unknown (as is usually the case) a Students t-test should be used. Note: the sample mean distribution must be approximately normal. That is, if n is small the population distribution should be normal where if n is large this is not required.

The p-value is calculated for the hypothesis is calculated using a t distribution with n - 1 degrees of freedom. The sample standard deviation **s** to approximate population standard deviation. Remember to use n - 1 in the denominator when calculating **s**.

This can be done using Scipy stats.ttest_1samp. The scipy method returns a tuple (test_statistic, p-value). The test statistic is the value on a standart t-distribution of speficied df and the p-value is for a two tailed t-test. A one tailed test is found by dividing simply dividing the p-value by two (that is, for the tail that might be interesting (ie. if x_bar = 1.1 and h0 was x > 1 it's not getting rejected, but if h0 was x < 1 dividing by two gives you the correct p-value)).

In [22]:
# Illowsky - Example 9.20
# right tailed t-test
data = [1.11, 1.07, 1.11, 1.07, 1.12, 1.08, 0.98, 0.98, 1.02, 0.95, 0.95]
h = 1.00 # null hypothesis: mean <= 1

# Using distribution explicitly (long way for explainatory purposes)
n = len(data)
x_bar = np.mean(data)
s = np.std(data, ddof=1) # sample std -> ddof = 1
pval = stats.t.sf(x_bar, n - 1, h, s / np.sqrt(n)) 
print('The p-value is {0:.4f}'.format(pval))

# The one-tail that's interesting is half the two-tailed
# * i.e. hypothesis mean <= 1 will never be rejected by a mean <= 1
pval = stats.ttest_1samp(data, h)[1] / 2
print('The p-value is {0:.4f} (using scipy method, dividing by two for 1 tailed test)'.format(pval))

# If you really want the one tail for the side that will never reject the null  
#print(1 - (stats.ttest_1samp(data, h)[1]) / 2) 

The p-value is 0.0359
The p-value is 0.0359 (using scipy method, dividing by two for 1 tailed test)


#### single population mean - known population standard deviation<a class="anchor" id="norm_1s"></a> 

When the population standard deviation is known (rarely the case) the p-value should be calculated using a normal distribution, sometimes called a z-test.

$$\bar{X} \sim N(\mu_X, \frac{\sigma_X}{{\sqrt{n}}})$$

In [43]:
# Illowsky - Example 9.9
#  null hypothesis mu >= h -> use left tail test
#  right tail test included for playing with inputs 
h = 15  
n = 10
x_bar = 17   # Sample mean
sigma = 0.5  # Known population standard deviation

pval_righttail = stats.norm.cdf(x_bar, h, (sigma / np.sqrt(n))) # Left tail test
#pval_lefttail = 1 - stats.norm.cdf(x_bar, h, (sigma / np.sqrt(n))) # Right tail test
pval_lefttail = stats.norm.sf(x_bar, h, (sigma / np.sqrt(n))) # More accurate right tail test

print('For the sample mean = {0}'.format(x_bar))
print('  If the null hypothesis was a population mean <= {0}, the p-value = {1:.4f}'\
         .format(h, pval_righttail))
print('  If the null hypothesis was a population mean >= {0}, the p-value = {1:.4f}'\
         .format(h, pval_lefttail))

# The z-test is not implemented in scipy. 
# The statsmodels module has an implementation, but it uses the sample standard deviation which
# defeats using it when the population standard deviation is known.

For the sample mean = 17
  If the null hypothesis was a population mean <= 15, the p-value = 1.0000
  If the null hypothesis was a population mean >= 15, the p-value = 0.0000


In [44]:
# Illowsky - Example 9.14 
#  null hypothesis mu <= h -> use right tail test
#  see previous example 9.9 for mu >= h
h = 16.43  
n = 15
x_bar = 16   # Sample mean
sigma = 0.8  # Known population standard deviation

pval_righttail = stats.norm.cdf(x_bar, h, (sigma / np.sqrt(n))) # Left tail test
pval_lefttail = stats.norm.sf(x_bar, h, (sigma / np.sqrt(n))) # More accurate right tail test

print('  For the null hypothesis was a population mean <= {0}, the p-value = {1:.4f}'\
         .format(h, pval_righttail))

  For the null hypothesis was a population mean <= 16.43, the p-value = 0.0187


#### Population proportion<a class="anchor" id="bi_1s"></a> 

Approximate as a normal distribution for a binomial divided by n. 
$$\large{P}' \sim N(p, \sqrt{\normalsize\frac{p q}{n}})$$

Note: Requires np > 5 and nq > 5 for an accurate result

In [4]:
# Illowsky - Example 9.17
# two-tailed

h = 0.50  # hypothesis proportion

n = 100   # number of samples
p = 0.53  # proportion observed

sigma = np.sqrt(h * (1 - h) / n)  # for proportion from binomial

# explainatory path to p-value 
if h < p:
    right_tail = stats.norm.sf(p, h, sigma)
    left_tail = stats.norm.cdf(h + (h - p), h, sigma)
else:
    right_tail = stats.norm.sf(h + (h - p), h, sigma)
    left_tail = stats.norm.cdf(p, h, sigma)
pval = right_tail + left_tail
print(pval)

# one line to p-value       
pval = 2 * stats.norm.sf(h + np.abs(h - p), h, sigma)
    
print(pval)

0.5485062355001469
0.5485062355001469


### Hypothesis testing with two independent samples<a class="anchor" id="twosamp_ind"></a> 

#### Welch<a class="anchor" id="welch"></a> 

In [5]:
# Two populations

n_x = 9      # number of x
x_bar = 2    # average for x
s_x = 0.866  # sample sigma for x

n_y = 16     # number of y
y_bar = 3.2  # average for y
s_y = 1      # sample sigma for y

# DOF - Welch Test
dof = ((s_x ** 2 / n_x + s_y ** 2 / n_y) ** 2 / 
      (1 / (n_x - 1) * (s_x ** 2 / n_x) ** 2 + 
       1 / (n_y - 1) * (s_y ** 2 / n_y) ** 2))
print(dof)

# Standard Error
std_err = np.sqrt((s_x ** 2 / n_x) + (s_y ** 2 / n_y))

pval = (stats.t.sf(np.abs(x_bar - y_bar), dof, 0, std_err) +
        stats.t.cdf(-1 * np.abs(x_bar - y_bar), dof, 0, std_err))

print(pval)

# If the data is available scipy.stats.ttest_ind(a, b, equal_var=False) can be used
# Note: if equal_var: perform a student's independent 2 sample test 
#       (assumes equal population sizes and variances) 
#       if not equal_var: perform Welch’s t-test

18.84659125336577
0.005401921297382211


#### right tailed dist + Cohen's d<a class="anchor" id="cohen_d"></a> 

In [6]:
# h0 mean of X less than mean of Y # right tailed distribution   

n_x = 11
x_bar = 4
s_x = 1.5

n_y = 9
y_bar = 3.5
s_y = 1

# DOF - Welch Test
dof = ((s_x ** 2 / n_x + s_y ** 2 / n_y) ** 2 / 
      (1 / (n_x - 1) * (s_x ** 2 / n_x) ** 2 + 
       1 / (n_y - 1) * (s_y ** 2 / n_y) ** 2))

std_err = np.sqrt(s_x ** 2 / n_x + s_y ** 2 / n_y)

pval = (stats.t.sf(x_bar - y_bar, dof, 0, std_err))

print(pval)

# cohen's d

s_pooled = np.sqrt((((n_x - 1) * s_x ** 2) +
                    ((n_y - 1) * s_y ** 2)) / 
                   (n_x + n_y - 2))

cohen_d = (x_bar - y_bar) / s_pooled

print(cohen_d)


0.1928185434187067
0.3841106397986879


#### Two Independent Population Proportions<a class="anchor" id="ind_pro"></a> 

In [7]:
x_a = 20
n_a = 200
x_b = 12
n_b = 200

p_a = (x_a / n_a)
p_b = (x_b / n_b)
p_c = (x_a + x_b) / (n_a + n_b)  # pooled proportion

sigma = np.sqrt(p_c * (1 - p_c) * (1 / n_a + 1 / n_b))

if p_a < p_b:
    p_a, p_b = p_b, p_a
p_val = (stats.norm.sf((p_a - p_b), 0, sigma) +
         stats.norm.cdf(0 - (p_a - p_b), 0, sigma))

print(p_val)

# not sure you can one line this with scipy, but probably with stats models


0.1403686607716731


### Hypothesis testing with two dependent samples<a class="anchor" id="twosamp_dep"></a>

In [8]:
data_before = [6.6, 6.5, 9.0, 10.3, 11.3, 8.1, 6.3, 11.6]
data_after =  [6.8, 2.4, 7.4,  8.5,  8.1, 6.1, 3.4,  2.0]

n = len(data_before)

before_array = np.array(data_before)
after_array = np.array(data_after)
diff_array = after_array - before_array

print(diff_array)

x_bar_diff = diff_array.mean()
s_diff = diff_array.std(ddof=1)
print(x_bar_diff, s_diff)

p_val = stats.t.cdf(x_bar_diff, n - 1, 0, s_diff / np.sqrt(n))

print(p_val)

# One line scipy stats method
#   divide p-val by two for one tailed test
stats.ttest_rel(data_after, data_before)[1] / 2


[ 0.2 -4.1 -1.6 -1.8 -3.2 -2.  -2.9 -9.6]
-3.1250000000000004 2.911430674329817
0.009477987786306367


0.009477987786306376

In [9]:
# right tailed

data_before = [205, 241, 338, 368]
data_after  = [295, 252, 330, 360]

n = len(data_before)

before_array = np.array(data_before)
after_array = np.array(data_after)
diff_array = after_array - before_array

print(diff_array)

x_bar_diff = diff_array.mean()
s_diff = diff_array.std(ddof=1)

p_val = stats.t.sf(x_bar_diff, n - 1, 0, s_diff / np.sqrt(n))

print(p_val)

print(stats.ttest_rel(data_after, data_before)[1] / 2) # divide by two for one tailed test 
print(stats.ttest_rel(data_before, data_after)[1] / 2) # ttest_rel tests the hypothesis that might be rejected



[90 11 -8 -8]
0.2149441957535246
0.2149441957535246
0.2149441957535246


### Chi-squared test<a class="anchor" id="chi_sq"></a> 

#### Goodness-of-fit<a class="anchor" id="chi_fit"></a> 

In [10]:
exp_dist = [12, 12, 12, 12, 12]  # Expected distribution
obs_dist = [15, 12,  9,  9, 15]  # Oberved distribution

dof = len(exp_dist) - 1

exp = np.array(exp_dist)
obs = np.array(obs_dist)

chi_stat = sum(((obs - exp) ** 2) / exp)

p_val = stats.chi2.sf(chi_stat, dof)

print(p_val)


# one line scipy stats method
#   dof defaults to k - 1, method accepts ddof argument which will set dof to k - 1 - ddof
print(stats.chisquare(obs_dist, exp_dist))



0.5578254003710748
Power_divergenceResult(statistic=3.0, pvalue=0.5578254003710748)


#### Independence<a class="anchor" id="chi_ind"></a> 

In [11]:
data = np.array([[111, 96, 48],
                 [96, 133, 61],
                 [91, 150, 53]])

n_terms = data.sum()
n_rows = len(data[:, 0])
n_cols = len(data[0, :])
dof = (n_rows - 1) * (n_cols - 1)

# Note - there's probably a better way to do this
expected = np.array([[(data[:, j].sum() * data[i, :].sum()) / n_terms 
                      for j in range(n_cols)] for i in range(n_rows)])

chi_stat = (((data - expected) ** 2) / expected).sum()

p_val = stats.chi2.sf(chi_stat, dof)

print(expected)
print(chi_stat)
print(p_val)
print()

# scipy stats method
print(stats.chi2_contingency(data))

[[ 90.57210965 115.19070322  49.23718713]
 [103.00357569 131.0011919   55.99523242]
 [104.42431466 132.80810489  56.76758045]]
12.990918513170868
0.011320253054188366

(12.990918513170868, 0.011320253054188366, 4, array([[ 90.57210965, 115.19070322,  49.23718713],
       [103.00357569, 131.0011919 ,  55.99523242],
       [104.42431466, 132.80810489,  56.76758045]]))


#### Homogeneity<a class="anchor" id="chi_homo"></a> 

This is the test for independence used to determine the probability that two populations have the same distribution.

In [12]:
data = np.array([[72, 84, 49, 45],
                 [91, 86, 88, 35]])

n_terms = data.sum()
n_rows = 2  # Should always be two here
n_cols = len(data[0, :])
dof = n_cols - 1

# Note - there's probably a better way to do this
expected = np.array([[(data[:,i].sum() * data[j,:].sum()) / n_terms 
                      for i in range(n_cols)] for j in range(n_rows)])

chi_stat = (((data - expected) ** 2) / expected).sum()

p_val = stats.chi2.sf(chi_stat, dof)

print(expected)
print(chi_stat)
print(p_val)
print()

[[74.09090909 77.27272727 62.27272727 36.36363636]
 [88.90909091 92.72727273 74.72727273 43.63636364]]
10.128696811826693
0.017503254828611012



#### Test of a single variance<a class="anchor" id="chi_var"></a> 

In [13]:
n = 25
s = 3.5
h = 7.2

dof = n - 1

chi_stat = ((n - 1) * s ** 2) / h ** 2

print(chi_stat)

p_val = stats.chi2.cdf(chi_stat, dof)  # left tailed

print(p_val)

5.671296296296296
4.213272184766762e-05


In [14]:
# increasing n increase test statistic, but decreases p-val for both left and right handed tests,
# guessing this is related to the change in distribution with dof?

n = 50
s = 3.5
h = 7.2

dof = n - 1

chi_stat = ((n - 1) * s ** 2) / h ** 2

print(chi_stat)

p_val = stats.chi2.cdf(chi_stat, dof)  # left tailed

print(p_val)

11.57889660493827
6.1840735576337476e-09


In [15]:
n = 25
s = 7.2
h = 3.5

dof = n - 1

chi_stat = ((n - 1) * s ** 2) / h ** 2

print(chi_stat)

p_val = stats.chi2.sf(chi_stat, dof)  # right tailed

print(p_val)

101.56408163265307
1.623621440614767e-11


In [16]:
n = 50
s = 7.2
h = 3.5

dof = n - 1

chi_stat = ((n - 1) * s ** 2) / h ** 2

print(chi_stat)

p_val = stats.chi2.sf(chi_stat, dof)  # right tailed

print(p_val)

207.36
2.2435449579324275e-21


### Test for correlation coefficient significance
Tests null hypothesis that the population correlation coefficient is 0

In [17]:
x = [ 65,  67,  71,  71,  66,  75,  67,  70,  71,  69,  69]
y = [175, 133, 185, 163, 126, 198, 153, 163, 159, 151, 159]

r, p_val = stats.pearsonr(x, y)
print('r = {0:.4f}, r^2 = {1:.4f}, p_val = {2:.4f}'.format(r, r ** 2, p_val))

r = 0.6631, r^2 = 0.4397, p_val = 0.0262


### ANOVA

## Sources<a class="anchor" id="sources"></a>

Illowsky, Barbara; Dean, Susan. Introductory Statistics. OpenStax College. Kindle Edition
https://openstax.org/details/introductory-statistics

SciPy 1.0.0 Release Notes: https://docs.scipy.org/doc/scipy/reference/index.html